In [27]:
# Import necessary libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report  # Add this line
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [15]:
# Set the path to the dataset directory
dataset_path = "C:/Users/sunny/Downloads/archive/leapGestRecog/leapGestRecog/00"

In [16]:
# Load images and labels
X = []
y = []

for gesture_label in os.listdir(dataset_path):
    gesture_path = os.path.join(dataset_path, gesture_label)
    
    for filename in os.listdir(gesture_path):
        img_path = os.path.join(gesture_path, filename)
        
        # if image file exists
        if os.path.isfile(img_path):
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # if the image loaded successfully
            if image is not None:
                image = cv2.resize(image, (64, 64))
                X.append(image)
                y.append(gesture_label)
            else:
                print(f"Error loading image: {img_path}")
        else:
            print(f"File not found: {img_path}")


In [17]:
# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

In [18]:
# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y)

In [19]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Build the model (same as before)
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [21]:
# Compile the model (same as before)
model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train.reshape(-1, 64, 64, 1))

In [24]:
# Train the model
history = model.fit(datagen.flow(X_train.reshape(-1, 64, 64, 1), y_train, batch_size=32), epochs=10, validation_data=(X_test.reshape(-1, 64, 64, 1), y_test))

Epoch 1/10


50/50 [==============================] - 4s 57ms/step - loss: 5.4398 - accuracy: 0.1494 - val_loss: 2.0643 - val_accuracy: 0.2875
Epoch 2/10
50/50 [==============================] - 2s 49ms/step - loss: 1.8846 - accuracy: 0.3262 - val_loss: 0.8883 - val_accuracy: 0.7525
Epoch 3/10
50/50 [==============================] - 2s 43ms/step - loss: 1.1225 - accuracy: 0.6156 - val_loss: 0.3838 - val_accuracy: 0.9125
Epoch 4/10
50/50 [==============================] - 2s 43ms/step - loss: 0.7107 - accuracy: 0.7431 - val_loss: 0.1579 - val_accuracy: 0.9450
Epoch 5/10
50/50 [==============================] - 2s 43ms/step - loss: 0.4112 - accuracy: 0.8669 - val_loss: 0.1964 - val_accuracy: 0.9425
Epoch 6/10
50/50 [==============================] - 2s 43ms/step - loss: 0.2762 - accuracy: 0.9087 - val_loss: 0.1790 - val_accuracy: 0.9275
Epoch 7/10
50/50 [==============================] - 2s 43ms/step - loss: 0.2560 - accuracy: 0.9206 - val_loss: 0.0296 - val_accuracy: 0.9950
Epoch 8/10


In [28]:
# Evaluate the model
y_pred = model.predict(X_test.reshape(-1, 64, 64, 1))
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy}")

13/13 [==============================] - 0s 16ms/step
Accuracy: 0.995


In [29]:
# Classification report
print(classification_report(y_true, y_pred_classes))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       0.96      1.00      0.98        47
           2       1.00      1.00      1.00        46
           3       1.00      1.00      1.00        36
           4       1.00      1.00      1.00        34
           5       1.00      0.97      0.99        34
           6       1.00      1.00      1.00        33
           7       1.00      0.98      0.99        44
           8       1.00      1.00      1.00        48
           9       1.00      1.00      1.00        42

    accuracy                           0.99       400
   macro avg       1.00      0.99      1.00       400
weighted avg       1.00      0.99      1.00       400

